# Étape 2 - Effectuez le pré-processing des données Open Agenda
#### Description
Récupérer les données d'événements à partir de la plateforme Open Agenda, les filtrer par localisation et période, et structurer ces données pour une utilisation future dans la base de données vectorielle.

## Librairies nécessaires

In [22]:
# Imports standards
import faiss
import requests
import pandas as pd
import numpy as np

# Imports LangChain
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings # ou SentenceTransformerEmbeddings
from sentence_transformers import SentenceTransformer


# Import Mistral
from mistralai.client import MistralClient

import os
from dotenv import load_dotenv
from mistralai import Mistral

## Récupération des données via l'API d'Open Agenda
Choix de la ville de Paris afin d'avoir beaucoup d'évènements différents. Choix de prendre 800 lignes pour le moment (raisonnable pour un poc). On décide de prendre des évènements qui se finissent après le 31/12/2024 afin d'avoir des informations sur la dernière année écoulée. Même si forcément nous aurons des informations sur des événements qui ont commencé en 2024 mais finissent bien en 2025. On a également ajouté la notion de firstdate_begin jusqu'au 01-01-2027 afin d'avoir des évènements à venir.

In [2]:
# Chemin pour l'API d'Open Agenda sur le dataset des événements publics
url = "https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/evenements-publics-openagenda/records"

# liste pour récupérer l'ensemble des lignes extraites de l'API
all_results = []

# Boucle sur offset pour récupérer plus de lignes que la limite imposée à 100
for offset in [0, 100, 200, 300, 400, 500, 600, 700]:
    params = {
        "limit": 100,
        "offset": offset,
        "lang": "fr",
        # Choix de la localisation à Paris
        "refine": ["location_city:Paris"],
        "where": (
            # Choix de prendre des évènements qui finissent après le 31/12/2024
            "lastdate_end >= '2024-12-31' "
            # Prendre les événements en  compte jusque début 2027
            "AND firstdate_begin <= '2027-01-01'"
        )
    }

    response = requests.get(url, params=params)
    response.raise_for_status()
    data = response.json()

    all_results.extend(data.get("results", []))

df = pd.DataFrame(all_results)
df.head()

,uid,slug,canonicalurl,title_fr,description_fr,longdescription_fr,conditions_fr,keywords_fr,image,imagecredits,...,originagenda_uid,contributor_email,contributor_contactnumber,contributor_contactname,contributor_contactposition,contributor_organization,category,country_fr,registration,links
0,83413776,carrefour-numerique_390,https://openagenda.com/cite-des-sciences/event...,Carrefour numérique,Le Carrefour numérique² un espace collaboratif...,"<p>Le Carrefour numérique², micro-lieu au sein...",Activités en accès libre ou sur inscription ou...,"[numérique, carrefour]",https://cibul.s3.amazonaws.com/3f28a89e0c83418...,None,...,76126842,None,None,None,None,None,None,France (Métropole),None,None
1,54724798,session-de-formation-croupier-via-poei-pour-le...,https://openagenda.com/semaine-des-metiers-du-...,Session de formation Croupier via POEI pour le...,Il s'agit d'une information collective autour ...,<p>Il s'agit d'une information collective auto...,None,"[Réunion d'information, La semaine du Tourisme...",None,None,...,38495884,None,None,None,None,None,None,France (Métropole),"[{""type"": ""link"", ""value"": ""https://meseveneme...",None
2,41364251,abya-yala-conte-amazonien,https://openagenda.com/hormur-plateforme-deven...,Abya Yala - Conte amazonien,La racontée Abya Yala est une mosaïque de pers...,<p>La racontée Abya Yala est une mosaïque de p...,16/12/8,"[conte, conte musical, spectacle, tout public,...",https://cdn.openagenda.com/main/53d7bfb771204d...,None,...,35234157,None,None,None,None,None,None,France (Métropole),"[{""type"": ""link"", ""value"": ""https://www.centre...",None
3,91172459,culture-autrement-atelier-creation-sonore-mon-...,https://openagenda.com/eteculturel-2025-ile-de...,Culture Autrement - atelier : Création sonore ...,Culture Autrement crée et anime des rencontres...,<p>Culture Autrement crée et anime des rencont...,Inscription directement sur place,None,https://cdn.openagenda.com/main/de9489f1b6124a...,© Les Agents Réunis,...,7191614,None,None,None,None,None,None,France (Métropole),None,None
4,48326082,festival-les-traverses-4461089,https://openagenda.com/eteculturel-2025-ile-de...,Festival Les Traverses,Festival Les Traverses - Samedi 28 juin,<p><strong>Festival</strong> <em><strong>Les T...,"Gratuit, en extérieur",None,https://cdn.openagenda.com/main/4d89f487902f42...,© Théâtre aux Mains Nues / Fabrication Maison,...,7191614,None,None,None,None,None,None,France (Métropole),None,None


### Regardons d'un peu plus près notre dataframe
#### On va déjà regarder si le nombre de lignes correspond bien à ce que l'on voulait

In [3]:
df_shape = df.shape
print(f"Nous avons {df_shape[0]} lignes et {df_shape[1]} colonnes dans notre dataframe.")

Nous avons 800 lignes et 56 colonnes dans notre dataframe.


#### Regardons le nombre de valeurs manquantes par variable

In [4]:
df.isnull().mean()

uid                            0.00000
slug                           0.00000
canonicalurl                   0.00000
title_fr                       0.00000
description_fr                 0.00000
longdescription_fr             0.02750
conditions_fr                  0.41750
keywords_fr                    0.75625
image                          0.03375
imagecredits                   0.18750
thumbnail                      0.03375
originalimage                  0.03375
updatedat                      0.00000
daterange_fr                   0.00000
firstdate_begin                0.00000
firstdate_end                  0.00000
lastdate_begin                 0.00000
lastdate_end                   0.00000
timings                        0.00000
accessibility                  0.70875
accessibility_label_fr         0.70875
location_uid                   0.00000
location_coordinates           0.00000
location_name                  0.00000
location_address               0.00000
location_district        

#### Filtrons sur les variables avec plus de 70% de valeurs manquantes afin de voir si les variables sont importantes ou non.

In [5]:
df_missed_values = df.loc[:, df.isnull().mean()>=0.70]
df_missed_values.isnull().mean()

keywords_fr                    0.75625
accessibility                  0.70875
accessibility_label_fr         0.70875
location_links                 0.83250
onlineaccesslink               0.97750
age_min                        0.89875
age_max                        0.87375
contributor_email              1.00000
contributor_contactnumber      1.00000
contributor_contactname        1.00000
contributor_contactposition    1.00000
contributor_organization       1.00000
category                       1.00000
links                          0.81750
dtype: float64

#### Regardons à quoi ressemble ces données

In [6]:
df_missed_values

,keywords_fr,accessibility,accessibility_label_fr,location_links,onlineaccesslink,age_min,age_max,contributor_email,contributor_contactnumber,contributor_contactname,contributor_contactposition,contributor_organization,category,links
0,"[numérique, carrefour]",mi;hi;vi,"[handicap moteur, handicap auditif, handicap v...",https://www.facebook.com/Cite.des.sciences/;ht...,None,15.0,NaN,None,None,None,None,None,None,None
1,"[Réunion d'information, La semaine du Tourisme...",None,None,None,None,NaN,NaN,None,None,None,None,None,None,None
2,"[conte, conte musical, spectacle, tout public,...",None,None,https://twitter.com/theatremandapa;https://www...,None,NaN,NaN,None,None,None,None,None,None,None
3,None,None,None,None,None,NaN,NaN,None,None,None,None,None,None,None
4,None,None,None,None,None,NaN,NaN,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,None,mi,[handicap moteur],https://www.facebook.com/monnaiedeparis;https:...,None,NaN,NaN,None,None,None,None,None,None,None
796,None,None,None,None,None,NaN,NaN,None,None,None,None,None,None,None
797,None,None,None,None,None,NaN,NaN,None,None,None,None,None,None,None
798,None,None,None,https://twitter.com/iptheologie,None,NaN,NaN,None,None,None,None,None,None,None


#### Au vu du nombre de données manquantes et la non-utilité pour notre chatbot (pas d'éléments concernant réellement un évènement particulier) on peut supprimer ces colonnes.

In [7]:
df_col_missed = df_missed_values.columns
df_col_missed

Index(['keywords_fr', 'accessibility', 'accessibility_label_fr',
       'location_links', 'onlineaccesslink', 'age_min', 'age_max',
       'contributor_email', 'contributor_contactnumber',
       'contributor_contactname', 'contributor_contactposition',
       'contributor_organization', 'category', 'links'],
      dtype='object')

#### Suppression des colonnes manquantes

In [8]:
df_firstclean = df.drop(columns=df_col_missed)

In [9]:
df_firstclean_shape = df_firstclean.shape
print(f"Nous avons {df_shape[0]} lignes et {df_shape[1]} colonnes dans notre dataframe.")
print(f"Nous avons {df_firstclean_shape[0]} lignes et {df_firstclean_shape[1]} colonnes après suppression.")

Nous avons 800 lignes et 56 colonnes dans notre dataframe.
Nous avons 800 lignes et 42 colonnes après suppression.


In [10]:
df_firstclean.isnull().mean()

uid                        0.00000
slug                       0.00000
canonicalurl               0.00000
title_fr                   0.00000
description_fr             0.00000
longdescription_fr         0.02750
conditions_fr              0.41750
image                      0.03375
imagecredits               0.18750
thumbnail                  0.03375
originalimage              0.03375
updatedat                  0.00000
daterange_fr               0.00000
firstdate_begin            0.00000
firstdate_end              0.00000
lastdate_begin             0.00000
lastdate_end               0.00000
timings                    0.00000
location_uid               0.00000
location_coordinates       0.00000
location_name              0.00000
location_address           0.00000
location_district          0.06750
location_insee             0.01500
location_postalcode        0.00500
location_city              0.00000
location_department        0.08875
location_region            0.00000
location_countrycode

#### Vérification des doublons

In [11]:
df_firstclean["uid"].duplicated().any()

np.False_

- Pas de doublons par identifiant

#### Vérification d'une date de début et de fin

In [12]:
df_firstclean[['uid','lastdate_end']].min()

uid                              10054610
lastdate_end    2025-01-11T17:00:00+00:00
dtype: object

In [13]:
df_firstclean.loc[df_firstclean['uid']=='10054610']

,uid,slug,canonicalurl,title_fr,description_fr,longdescription_fr,conditions_fr,image,imagecredits,thumbnail,...,location_website,location_tags,location_description_fr,location_access_fr,attendancemode,status,originagenda_title,originagenda_uid,country_fr,registration
438,10054610,visite-dun-jardin-tinctorial-avec-demo-teinture,https://openagenda.com/rdvj-2025-ile-de-france...,Visite d'un jardin tinctorial avec démo teinture,Nous vous faisons découvrir notre jardin tinct...,<p>Nous vous faisons découvrir notre jardin ti...,"gratuit, places limitées",https://cdn.openagenda.com/main/593c38fc576945...,©WHOLE,https://cdn.openagenda.com/main/593c38fc576945...,...,http://www.whole.fr,Ouverture exceptionnelle;Jardin de création ré...,Jardin Parisculteurs spécialisé en plantes tin...,Près du M°14 Maison Blanche ou M°7 Porte d'Ita...,"{""id"": 1, ""label"": {""fr"": ""Sur place"", ""en"": ""...","{""id"": 1, ""label"": {""fr"": ""Programm\u00e9"", ""e...",Rendez-vous aux jardins 2025 : Île-de-France,4144624,France (Métropole),"[{""type"": ""link"", ""value"": ""https://whole.fr/p..."


- Vérification si c'est cohérent et ça l'est car l'évènement était bien présent en 2025.

In [14]:
df_firstclean[['uid','firstdate_begin']].min()

uid                                 10054610
firstdate_begin    2023-01-24T09:00:00+00:00
dtype: object

In [15]:
df_firstclean.loc[df_firstclean['uid']=='99807144']

,uid,slug,canonicalurl,title_fr,description_fr,longdescription_fr,conditions_fr,image,imagecredits,thumbnail,...,location_website,location_tags,location_description_fr,location_access_fr,attendancemode,status,originagenda_title,originagenda_uid,country_fr,registration
271,99807144,escape-game-de-lindustrie-2617574,https://openagenda.com/semaine-industrie-2025/...,Escape Game de l'Industrie,Un Escape Game sur table où chaque équipe coll...,<p>Aidez l’entreprise Greenlight à recevoir le...,None,https://cdn.openagenda.com/main/0f6772036e7048...,None,https://cdn.openagenda.com/main/0f6772036e7048...,...,None,None,None,None,"{""id"": 1, ""label"": {""fr"": ""Sur place"", ""en"": ""...","{""id"": 1, ""label"": {""fr"": ""Programm\u00e9"", ""e...",Semaine de l'industrie 2025,9464342,France (Métropole),"[{""type"": ""link"", ""value"": ""https://lusineephe..."


#### On ajoute les éléments importants dans un seule colonne texte afin de réaliser les embeddings. On en profite pour rendre un peu plus propre l'affichage de nos varibales, on ajoute /n pour mettre les éléments à la ligne.

In [16]:
df_firstclean["text_for_embedding"] = (
    df_firstclean["title_fr"] + "\n" +
    df_firstclean["description_fr"].fillna("") + "\n" +
    df_firstclean["longdescription_fr"].fillna("") + "\n" +
    df_firstclean["location_name"].fillna("") + "\n" +
    df_firstclean["location_city"].fillna("")
)

#### Regardons à quoi cela peut ressembler

In [17]:
df_firstclean["text_for_embedding"]

0      Carrefour numérique\nLe Carrefour numérique² u...
1      Session de formation Croupier via POEI pour le...
2      Abya Yala - Conte amazonien\nLa racontée Abya ...
3      Culture Autrement - atelier : Création sonore ...
4      Festival Les Traverses\nFestival Les Traverses...
                             ...                        
795    Visite flash du musée de la Monnaie de Paris\n...
796    La vie cachée des œuvres. Rencontre-performanc...
797    Conférence à la Maison de l'Italie sur le chan...
798    Visite de la Faculté de théologie protestante ...
799    SOUNDINITIATIVE : CONCERT #2 DU CURSUS IRCAM\n...
Name: text_for_embedding, Length: 800, dtype: object

#### Testons deux manières d'obtenir les embeddings

In [19]:
# Chargement du modèle SBERT
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Textes à vectoriser
df = df_firstclean["text_for_embedding"]
# Obtention des embeddings
embeddings = model.encode(df)

print(embeddings)

[[-0.07666893 -0.03182433 -0.05550356 ... -0.03910216  0.06446592
  -0.07069877]
 [-0.10829046  0.04042959 -0.06553445 ...  0.04624669 -0.00930399
  -0.04157751]
 [ 0.00363422 -0.00085042 -0.04936273 ...  0.0223415   0.00664138
  -0.05324589]
 ...
 [ 0.0399245   0.01874438  0.0496708  ...  0.02158543 -0.02996898
   0.03084538]
 [-0.04316112  0.01043762 -0.00165652 ... -0.00159707  0.04047303
   0.0172886 ]
 [-0.01152389 -0.06214816  0.01804776 ...  0.06089856 -0.04568607
  -0.00428984]]


#### Nous avons bien la génération de nos vecteurs

#### Pour mesurer la similarité entre deux embeddings, on utilise la similarité cosinus :

In [21]:
# Vecteurs des deux textes
vec1 = embeddings[0]
vec2 = embeddings[1]

# Calcul de la similarité cosinus
similarité = np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

print(f"Similarité : {similarité}")

Similarité : 0.2784297466278076


- Le plus haut score étant 1 et le plus petit étant 0, on voit que le résultat n'est pas très bon, et ne trouve pas beaucoup de similarité entre nos deux phrases.

#### Extraction avec Mistral
- Nous allons maintenant exploiter les modèles d'embedding de Mistral, accessibles via des API serverless. Cette approche simplifie grandement notre démarche.


In [ ]:
# On charge notre clé API Mistral depuis env
load_dotenv()

# On active notre client mistral
client = Mistral(
    api_key=os.getenv("MISTRAL_KEY")
)

# On prend 50 textes car nous avons un nombre limité de token
texts = df_firstclean["text_for_embedding"].astype(str).tolist()[:50]

response = client.embeddings.create(
    model="mistral-embed",
    inputs=texts
)

# TOUS les embeddings
vectors = [item.embedding for item in response.data]

print("Nombre d'embeddings :", len(vectors))
print("Dimension :", len(vectors[0]))
print("Aperçu :", vectors[0][:5])


Nombre d'embeddings : 50
Dimension : 1024
Aperçu : [-0.0167999267578125, 0.011016845703125, 0.031646728515625, -0.0022335052490234375, 0.03631591796875]


#### Test de similarité

In [30]:
# Embeddings de deux événements différents
vec1 = np.array(vectors[0])
vec2 = np.array(vectors[1])

similarite = np.dot(vec1, vec2) / (
    np.linalg.norm(vec1) * np.linalg.norm(vec2)
)

print("Similarité :", similarite)


Similarité : 0.7461442977967905


- Cette méthode d'extraction nous permet d'avoir un meilleur résultat. Les deux textes sont sémantiquements plus proches que le premier modèle.